In [25]:
import boto3
import sagemaker
import pandas as pd

from glob import glob

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [10]:
%store -r processed_train_data_s3_uri
%store -r processed_validation_data_s3_uri
%store -r processed_test_data_s3_uri
%store -r max_seq_length
%store -r experiment_name
%store -r trial_name

In [11]:
max_seq_length

183

### Initial Hyperparameters:

In [7]:
epochs = 3
learning_rate = 0.00001
epsilon = 0.00000001
train_batch_size = 128
validation_batch_size = 128
test_batch_size = 128
train_steps_per_epoch = 100
validation_steps = 100
test_steps = 100
train_instance_count = 1
train_instance_type = "ml.c5.9xlarge"
train_volume_size = 1024
use_xla = True
use_amp = True
freeze_bert_layer = False
enable_sagemaker_debugger = False
enable_checkpointing = False
enable_tensorboard = True
input_mode = "File"
run_validation = True
run_test = True
run_sample_predictions = True

### Checkpoint Location

In [8]:
import uuid

checkpoint_s3_prefix = "checkpoints/{}".format(str(uuid.uuid4()))
checkpoint_s3_uri = "s3://{}/{}/".format(bucket, checkpoint_s3_prefix)

print(checkpoint_s3_uri)

s3://sagemaker-us-east-1-211125778552/checkpoints/da9d1272-91ad-4215-a09a-b6b06eec406d/


### Metrics

In [9]:
metrics_definitions = [
    {"Name": "train:loss", "Regex": "loss: ([0-9\\.]+)"},
    {"Name": "train:accuracy", "Regex": "accuracy: ([0-9\\.]+)"},
    {"Name": "validation:loss", "Regex": "val_loss: ([0-9\\.]+)"},
    {"Name": "validation:accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"},
]

In [10]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point="tf_bert_reviews.py",
    source_dir="src",
    role=role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    volume_size=train_volume_size,
    checkpoint_s3_uri=checkpoint_s3_uri,
    py_version="py37",
    framework_version="2.3.1",
    hyperparameters={
        "epochs": epochs,
        "learning_rate": learning_rate,
        "epsilon": epsilon,
        "train_batch_size": train_batch_size,
        "validation_batch_size": validation_batch_size,
        "test_batch_size": test_batch_size,
        "train_steps_per_epoch": train_steps_per_epoch,
        "validation_steps": validation_steps,
        "test_steps": test_steps,
        "use_xla": use_xla,
        "use_amp": use_amp,
        "max_seq_length": max_seq_length,
        "freeze_bert_layer": freeze_bert_layer,
        "enable_sagemaker_debugger": enable_sagemaker_debugger,
        "enable_checkpointing": enable_checkpointing,
        "enable_tensorboard": enable_tensorboard,
        "run_validation": run_validation,
        "run_test": run_test,
        "run_sample_predictions": run_sample_predictions,
    },
    input_mode=input_mode,
    metric_definitions=metrics_definitions
)

### Training the BERT model

In [11]:
experiment_config = {"ExperimentName": experiment_name, "TrialName": trial_name, "TrialComponentDisplayName": "train"}

In [12]:
from sagemaker.inputs import TrainingInput

s3_input_train_data = TrainingInput(s3_data=processed_train_data_s3_uri, distribution="ShardedByS3Key")
s3_input_validation_data = TrainingInput(s3_data=processed_validation_data_s3_uri, distribution="ShardedByS3Key")
s3_input_test_data = TrainingInput(s3_data=processed_test_data_s3_uri, distribution="ShardedByS3Key")

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-211125778552/sagemaker-scikit-learn-2024-04-20-01-40-23-350/output/bert-train', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-211125778552/sagemaker-scikit-learn-2024-04-20-01-40-23-350/output/bert-validation', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-211125778552/sagemaker-scikit-learn-2024-04-20-01-40-23-350/output/bert-test', 'S3DataDistributionType': 'ShardedByS3Key'}}}


In [13]:
estimator.fit(
    inputs={"train": s3_input_train_data, "validation": s3_input_validation_data, "test": s3_input_test_data},
    experiment_config=experiment_config,
    wait=False,
)

In [55]:
!aws s3 ls s3://$bucket/tensorflow-training-2024-04-20-02-13-51-094/output/

2024-04-20 03:27:43  834782307 model.tar.gz


In [14]:
training_job_name = estimator.latest_training_job.name
print("Training Job Name:  {}".format(training_job_name))

Training Job Name:  tensorflow-training-2024-04-20-02-13-51-094


In [15]:
%%time

estimator.latest_training_job.wait(logs=True)

2024-04-20 02:13:53 Starting - Starting the training job...
2024-04-20 02:14:17 Starting - Preparing the instances for trainingProfilerReport-1713579231: InProgress
...
2024-04-20 02:14:57 Downloading - Downloading the training image...
2024-04-20 02:15:24 Training - Training image download completed. Training in progress....2024-04-20 02:15:43.159993: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-04-20 02:15:43.160133: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-04-20 02:15:43.188848: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-04-20 02:15:44,395 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2024-04-20 02:15:44,404 sagemaker-training-toolkit INFO     No GPUs detected (nor

In [16]:
estimator.training_job_analytics.dataframe()

,timestamp,metric_name,value
0,0.0,train:loss,2.301567
1,60.0,train:loss,2.301033
2,120.0,train:loss,2.299267
3,180.0,train:loss,2.297617
4,240.0,train:loss,2.296033
...,...,...,...
115,1320.0,validation:loss,2.254000
116,2640.0,validation:loss,2.245000
117,0.0,validation:accuracy,0.153800
118,1320.0,validation:accuracy,0.189300


### Hyperparameter Tuning

In [17]:
epochs = 1
learning_rate = 0.00001
epsilon = 0.00000001
train_batch_size = 128
validation_batch_size = 128
test_batch_size = 128
train_steps_per_epoch = 100
validation_steps = 100
test_steps = 100
train_instance_count = 1
train_instance_type = "ml.c5.9xlarge"
train_volume_size = 1024
use_xla = True
use_amp = True
freeze_bert_layer = False
enable_sagemaker_debugger = False
enable_checkpointing = False
enable_tensorboard = True
input_mode = "File"
run_validation = True
run_test = True
run_sample_predictions = True

In [18]:
from sagemaker.tuner import IntegerParameter
from sagemaker.tuner import ContinuousParameter
from sagemaker.tuner import CategoricalParameter
from sagemaker.tuner import HyperparameterTuner

hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.00001, 0.0001, scaling_type="Linear"),
    "train_batch_size": CategoricalParameter([128, 256]),
    "freeze_bert_layer": CategoricalParameter([True, False]),
}

In [51]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point="tf_bert_reviews.py",
    source_dir="src",
    role=role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    volume_size=train_volume_size,
    checkpoint_s3_uri=checkpoint_s3_uri,
    py_version="py37",
    framework_version="2.3.1",
    hyperparameters={
        "epochs": epochs,
        "learning_rate": learning_rate,
        "epsilon": epsilon,
        "train_batch_size": train_batch_size,
        "validation_batch_size": validation_batch_size,
        "test_batch_size": test_batch_size,
        "train_steps_per_epoch": train_steps_per_epoch,
        "validation_steps": validation_steps,
        "test_steps": test_steps,
        "use_xla": use_xla,
        "use_amp": use_amp,
        "max_seq_length": max_seq_length,
        "freeze_bert_layer": freeze_bert_layer,
        "enable_sagemaker_debugger": enable_sagemaker_debugger,
        "enable_checkpointing": enable_checkpointing,
        "enable_tensorboard": enable_tensorboard,
        "run_validation": run_validation,
        "run_test": run_test,
        "run_sample_predictions": run_sample_predictions,
    },
    input_mode=input_mode,
    metric_definitions=metrics_definitions
)

NameError: name 'train_instance_count' is not defined

In [20]:
objective_metric_name = "validation:accuracy"

tuner = HyperparameterTuner(
    estimator=estimator,
    objective_type="Maximize",
    objective_metric_name=objective_metric_name,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metrics_definitions,
    max_jobs=2,
    max_parallel_jobs=1,
    strategy="Bayesian",
    early_stopping_type="Auto",
)

In [21]:
tuner.fit(
    inputs={"train": s3_input_train_data, "validation": s3_input_validation_data, "test": s3_input_test_data},
    include_cls_metadata=False,
    wait=False,
)

In [22]:
from pprint import pprint

tuning_job_name = tuner.latest_tuning_job.job_name

In [23]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/hyper-tuning-jobs/{}">Hyper-Parameter Tuning Job</a></b>'.format(
            region, tuning_job_name
        )
    )
)

In [47]:
def load_checkpoint_model(checkpoint_path):
    import glob
    import os

    glob_pattern = os.path.join(checkpoint_path, "*.h5")
    print("glob pattern {}".format(glob_pattern))

    list_of_checkpoint_files = glob.glob(glob_pattern)
    print("List of checkpoint files {}".format(list_of_checkpoint_files))

    latest_checkpoint_file = max(list_of_checkpoint_files)
    print("Latest checkpoint file {}".format(latest_checkpoint_file))

    initial_epoch_number_str = latest_checkpoint_file.rsplit("_", 1)[-1].split(".h5")[0]
    initial_epoch_number = int(initial_epoch_number_str)

    loaded_model = TFDistilBertForSequenceClassification.from_pretrained(latest_checkpoint_file, config=config)

    print("loaded_model {}".format(loaded_model))
    print("initial_epoch_number {}".format(initial_epoch_number))

    return loaded_model, initial_epoch_number

In [50]:
load_checkpoint_model(checkpoint_s3_uri)

NameError: name 'checkpoint_s3_uri' is not defined